In [2]:
import tensorflow as tf
tf.__version__


'2.12.0'

In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [4]:
subject = 'Tocantins - Consumo de Cimento (t)'
split_index = 6 #Referente aos 7 anos de input  

In [5]:
data = pd.read_csv('../Inputs Atuais/2014_01_model_input_TO.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data

,Ano,Tocantins - PIB - Estadual,Tocantins - PIB - Construção Civil,Tocantins - PIB - Per Capita,Tocantins - PIB - Preços de Mercado,Tocantins - IDH,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),...,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Tocantins - IDH Renda,Tocantins - Desemprego,Tocantins - Consumo de Cimento (t)
0,2014,1.945461e+07,1.234495e+06,10.702690,2.301322e+07,0.724805,5.676413e+08,2.779340e+09,1.954361e+10,4.143397e+10,...,45.725826,978.988335,1363.511779,27.574024,18.568157,62.927666,1.510968,0.675455,10.192991,426.767286
1,2015,1.971544e+07,1.239338e+06,10.728749,2.324825e+07,0.735765,5.179392e+08,2.561731e+09,1.327596e+10,4.469706e+10,...,48.162288,978.983844,1381.825545,27.799714,18.471946,62.070670,1.531417,0.684629,10.501090,454.929429
2,2016,1.989200e+07,1.246862e+06,10.746325,2.347786e+07,0.736289,4.524144e+08,2.516859e+09,1.011370e+10,4.640544e+10,...,40.542577,978.957795,1390.699392,27.680489,18.345425,62.095581,1.511336,0.684323,10.392740,489.016286
3,2017,1.998196e+07,1.262231e+06,10.764439,2.371682e+07,0.736497,4.213689e+08,2.497710e+09,8.456309e+09,4.607888e+10,...,37.984321,979.036017,1384.229204,27.536429,18.412455,62.734632,1.476663,0.684807,9.894472,500.270000
4,2018,1.996897e+07,1.273956e+06,10.766724,2.394166e+07,0.736179,4.166580e+08,2.797145e+09,7.937760e+09,4.467878e+10,...,36.012234,979.117292,1377.691540,27.546480,18.670122,63.335663,1.430062,0.684608,9.761228,396.438000
5,2019,1.987589e+07,1.284682e+06,10.762429,2.418145e+07,0.735013,4.462625e+08,3.484252e+09,1.091282e+10,4.379096e+10,...,33.813157,978.997253,1382.278832,27.624887,18.623398,62.765394,1.377912,0.683390,9.617662,402.932000
6,2020,1.960866e+07,1.299452e+06,10.729158,2.439193e+07,0.733560,5.489412e+08,4.114911e+09,1.625333e+10,4.320994e+10,...,31.839958,978.864073,1376.895360,27.617263,18.457904,62.023537,1.251304,0.682517,9.371493,500.126000
7,2021,1.911763e+07,1.323080e+06,10.676667,2.457920e+07,0.731611,6.638811e+08,4.421848e+09,1.933031e+10,4.488070e+10,...,29.778041,978.924410,1372.511562,27.712744,18.223438,60.927060,1.222737,0.681745,9.066728,542.030000
8,2022,1.852731e+07,1.348819e+06,10.612114,2.477540e+07,0.731667,7.010332e+08,4.533216e+09,1.997888e+10,4.699319e+10,...,28.950766,979.006198,1375.916061,27.794396,18.022128,60.176598,1.267841,0.681540,9.118915,554.321000


In [6]:
input_data = data.iloc[:-2, 1:-1]
input_data

,Tocantins - PIB - Estadual,Tocantins - PIB - Construção Civil,Tocantins - PIB - Per Capita,Tocantins - PIB - Preços de Mercado,Tocantins - IDH,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),Contratação SFH (Bi R$),...,Tocantins - Produção de Cimento (t),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Tocantins - IDH Renda,Tocantins - Desemprego
0,1.945461e+07,1.234495e+06,10.702690,2.301322e+07,0.724805,5.676413e+08,2.779340e+09,1.954361e+10,4.143397e+10,4.570163e+10,...,534.422193,45.725826,978.988335,1363.511779,27.574024,18.568157,62.927666,1.510968,0.675455,10.192991
1,1.971544e+07,1.239338e+06,10.728749,2.324825e+07,0.735765,5.179392e+08,2.561731e+09,1.327596e+10,4.469706e+10,3.539769e+10,...,515.383793,48.162288,978.983844,1381.825545,27.799714,18.471946,62.070670,1.531417,0.684629,10.501090
2,1.989200e+07,1.246862e+06,10.746325,2.347786e+07,0.736289,4.524144e+08,2.516859e+09,1.011370e+10,4.640544e+10,3.407411e+10,...,535.186108,40.542577,978.957795,1390.699392,27.680489,18.345425,62.095581,1.511336,0.684323,10.392740
3,1.998196e+07,1.262231e+06,10.764439,2.371682e+07,0.736497,4.213689e+08,2.497710e+09,8.456309e+09,4.607888e+10,3.680756e+10,...,580.856957,37.984321,979.036017,1384.229204,27.536429,18.412455,62.734632,1.476663,0.684807,9.894472
4,1.996897e+07,1.273956e+06,10.766724,2.394166e+07,0.736179,4.166580e+08,2.797145e+09,7.937760e+09,4.467878e+10,4.727230e+10,...,599.811915,36.012234,979.117292,1377.691540,27.546480,18.670122,63.335663,1.430062,0.684608,9.761228
5,1.987589e+07,1.284682e+06,10.762429,2.418145e+07,0.735013,4.462625e+08,3.484252e+09,1.091282e+10,4.379096e+10,7.227332e+10,...,654.754815,33.813157,978.997253,1382.278832,27.624887,18.623398,62.765394,1.377912,0.683390,9.617662
6,1.960866e+07,1.299452e+06,10.729158,2.439193e+07,0.733560,5.489412e+08,4.114911e+09,1.625333e+10,4.320994e+10,9.809293e+10,...,735.747847,31.839958,978.864073,1376.895360,27.617263,18.457904,62.023537,1.251304,0.682517,9.371493


In [7]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0    454.929429
1    489.016286
2    500.270000
3    396.438000
4    402.932000
5    500.126000
6    542.030000
7    554.321000
8           NaN
Name: Tocantins - Consumo de Cimento (t), dtype: float64

In [8]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Tocantins - PIB - Estadual,Tocantins - PIB - Construção Civil,Tocantins - PIB - Per Capita,Tocantins - PIB - Preços de Mercado,Tocantins - IDH,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),Contratação SFH (Bi R$),...,Tocantins - Produção de Cimento (t),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Tocantins - IDH Renda,Tocantins - Desemprego
0,-1.967934,-1.227943,-1.839197,-1.464920,-2.221323,1.779974,0.019134,2.003355,-1.880579,0.034168,...,-0.747347,1.045989,-0.483413,-1.979981,-0.579269,0.458861,0.607178,0.707560,-2.214759,0.408614
1,-0.542907,-0.962851,-0.712450,-0.874689,0.400329,0.870373,-0.621387,0.401153,0.111653,-0.753105,...,-1.146491,1.522130,-0.569949,0.213980,1.888387,-0.375580,-1.300740,1.089232,0.526004,1.355460
2,0.421691,-0.550998,0.047506,-0.298049,0.525693,-0.328802,-0.753466,-0.407217,1.154682,-0.854233,...,-0.731332,0.033061,-1.071894,1.277052,0.584796,-1.472916,-1.245281,0.714434,0.434472,1.022479
3,0.913183,0.290347,0.830761,0.302046,0.575474,-0.896969,-0.809829,-0.830898,0.955301,-0.645384,...,0.226165,-0.466882,0.435367,0.501934,-0.990334,-0.891557,0.177430,0.067250,0.579029,-0.508792
4,0.842257,0.932140,0.929553,0.866705,0.499310,-0.983185,0.071542,-0.963455,0.100493,0.154175,...,0.623559,-0.852273,2.001471,-0.281268,-0.880431,1.343218,1.515497,-0.802554,0.519474,-0.918277
5,0.333711,1.519305,0.743828,1.468906,0.220517,-0.441390,2.094006,-0.202939,-0.441549,2.064379,...,1.775446,-1.282024,-0.311582,0.268283,-0.023150,0.937974,0.245916,-1.775923,0.155780,-1.359484


In [9]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0    454.929429
1    489.016286
2    500.270000
3    396.438000
4    402.932000
5    500.126000
Name: Tocantins - Consumo de Cimento (t), dtype: float64

In [10]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,Tocantins - PIB - Estadual,Tocantins - PIB - Construção Civil,Tocantins - PIB - Per Capita,Tocantins - PIB - Preços de Mercado,Tocantins - IDH,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),Contratação SFH (Bi R$),...,Tocantins - Produção de Cimento (t),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Tocantins - IDH Renda,Tocantins - Desemprego
6,-0.959434,1.618038,-0.62215,1.475914,-0.117545,1.16956,2.035192,0.985175,-0.705937,2.048732,...,1.948593,-1.306124,-1.803231,-0.345228,-0.098527,-0.452549,-1.149265,-2.063869,-0.097313,-1.529923


In [11]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

6    542.03
Name: Tocantins - Consumo de Cimento (t), dtype: float64

In [15]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=1000, verbose=1)
    # Método estocástico e learning rate=0.01
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(74, activation='tanh', input_shape=(74,)),
        tf.keras.layers.Dense(296, activation='tanh'),
        tf.keras.layers.Dense(296, activation='tanh'),
        tf.keras.layers.Dense(148, activation='tanh'),
        tf.keras.layers.Dense(37, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=5000, 
                      batch_size=len(train_input), 
                      validation_data=(train_input.iloc[-1:], train_target.iloc[-1:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [16]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(10)]
    print(random_seeds)

    best_val_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, 0, seed)

        current_val_loss = min(history.history['val_loss'])
        print(f"val_loss: {current_val_loss}")

        if current_val_loss < best_val_loss:
            best_val_loss = current_val_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [17]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[1751982521, 2632174883, 4148047056, 430706539, 3894850973, 4202634300, 759149213, 1615823428, 940955571, 2663516689]


Step: 0 ___________________________________________
Epoch 1741: early stopping
val_loss: 2361.776611328125
winner_seed: 1751982521


Step: 1 ___________________________________________
Epoch 1735: early stopping
val_loss: 2518.537109375


Step: 2 ___________________________________________
Epoch 1546: early stopping
val_loss: 4483.20703125


Step: 3 ___________________________________________
Epoch 2081: early stopping
val_loss: 4483.82861328125


Step: 4 ___________________________________________
Epoch 1654: early stopping
val_loss: 352.2941589355469
winner_seed: 3894850973


Step: 5 ___________________________________________
Epoch 4288: early stopping
val_loss: 2361.699462890625


Step: 6 ___________________________________________
Epoch 1116: early stopping
val_loss: 0.0008999267593026161
winner_seed: 759149213


Step: 7 __________________________________________

In [18]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/5000
1/1 [==============================] - 0s 277ms/step - loss: 211002.0469 - val_loss: 235496.2344
Epoch 2/5000
1/1 [==============================] - 0s 12ms/step - loss: 198019.5469 - val_loss: 332504.5000
Epoch 3/5000
1/1 [==============================] - 0s 12ms/step - loss: 159430.5469 - val_loss: 447168.2812
Epoch 4/5000
1/1 [==============================] - 0s 12ms/step - loss: 278453.8750 - val_loss: 248092.0625
Epoch 5/5000
1/1 [==============================] - 0s 12ms/step - loss: 201518.8906 - val_loss: 185000.5781
Epoch 6/5000
1/1 [==============================] - 0s 11ms/step - loss: 175390.5000 - val_loss: 159263.5938
Epoch 7/5000
1/1 [==============================] - 0s 12ms/step - loss: 159011.6094 - val_loss: 146286.4531
Epoch 8/5000
1/1 [==============================] - 0s 11ms/step - loss: 146263.2188 - val_loss: 138009.8438
Epoch 9/5000
1/1 [==============================] - 0s 11ms/step - loss: 135729.5469 - val_loss: 131556.9688
Epoch 10/5000
1/1 

Epoch 77/5000
1/1 [==============================] - 0s 11ms/step - loss: 10799.7178 - val_loss: 2714.5981
Epoch 78/5000
1/1 [==============================] - 0s 12ms/step - loss: 10464.4482 - val_loss: 2507.5344
Epoch 79/5000
1/1 [==============================] - 0s 12ms/step - loss: 10140.9463 - val_loss: 2313.1057
Epoch 80/5000
1/1 [==============================] - 0s 12ms/step - loss: 9828.7773 - val_loss: 2130.6868
Epoch 81/5000
1/1 [==============================] - 0s 12ms/step - loss: 9527.5273 - val_loss: 1959.6698
Epoch 82/5000
1/1 [==============================] - 0s 12ms/step - loss: 9236.7900 - val_loss: 1799.4757
Epoch 83/5000
1/1 [==============================] - 0s 12ms/step - loss: 8956.1797 - val_loss: 1649.5526
Epoch 84/5000
1/1 [==============================] - 0s 12ms/step - loss: 8685.3252 - val_loss: 1509.3799
Epoch 85/5000
1/1 [==============================] - 0s 12ms/step - loss: 8423.8643 - val_loss: 1378.4469
Epoch 86/5000
1/1 [========================

1/1 [==============================] - 0s 11ms/step - loss: 1507.4559 - val_loss: 339.7040
Epoch 155/5000
1/1 [==============================] - 0s 11ms/step - loss: 1478.5811 - val_loss: 348.8384
Epoch 156/5000
1/1 [==============================] - 0s 11ms/step - loss: 1450.4647 - val_loss: 357.7591
Epoch 157/5000
1/1 [==============================] - 0s 11ms/step - loss: 1423.0824 - val_loss: 366.4582
Epoch 158/5000
1/1 [==============================] - 0s 11ms/step - loss: 1396.4108 - val_loss: 374.9331
Epoch 159/5000
1/1 [==============================] - 0s 11ms/step - loss: 1370.4277 - val_loss: 383.1799
Epoch 160/5000
1/1 [==============================] - 0s 11ms/step - loss: 1345.1124 - val_loss: 391.1952
Epoch 161/5000
1/1 [==============================] - 0s 11ms/step - loss: 1320.4419 - val_loss: 398.9765
Epoch 162/5000
1/1 [==============================] - 0s 11ms/step - loss: 1296.3976 - val_loss: 406.5217
Epoch 163/5000
1/1 [==============================] - 0s 11ms

Epoch 232/5000
1/1 [==============================] - 0s 11ms/step - loss: 463.2902 - val_loss: 471.2363
Epoch 233/5000
1/1 [==============================] - 0s 11ms/step - loss: 457.8214 - val_loss: 468.2836
Epoch 234/5000
1/1 [==============================] - 0s 12ms/step - loss: 452.4456 - val_loss: 465.2929
Epoch 235/5000
1/1 [==============================] - 0s 11ms/step - loss: 447.1606 - val_loss: 462.2671
Epoch 236/5000
1/1 [==============================] - 0s 12ms/step - loss: 441.9643 - val_loss: 459.2067
Epoch 237/5000
1/1 [==============================] - 0s 12ms/step - loss: 436.8560 - val_loss: 456.1121
Epoch 238/5000
1/1 [==============================] - 0s 11ms/step - loss: 431.8325 - val_loss: 452.9890
Epoch 239/5000
1/1 [==============================] - 0s 11ms/step - loss: 426.8933 - val_loss: 449.8379
Epoch 240/5000
1/1 [==============================] - 0s 11ms/step - loss: 422.0354 - val_loss: 446.6564
Epoch 241/5000
1/1 [==============================] - 0

1/1 [==============================] - 0s 11ms/step - loss: 217.5867 - val_loss: 226.2496
Epoch 311/5000
1/1 [==============================] - 0s 11ms/step - loss: 215.9455 - val_loss: 223.6681
Epoch 312/5000
1/1 [==============================] - 0s 11ms/step - loss: 214.3282 - val_loss: 221.1086
Epoch 313/5000
1/1 [==============================] - 0s 11ms/step - loss: 212.7339 - val_loss: 218.5711
Epoch 314/5000
1/1 [==============================] - 0s 11ms/step - loss: 211.1620 - val_loss: 216.0554
Epoch 315/5000
1/1 [==============================] - 0s 11ms/step - loss: 209.6130 - val_loss: 213.5614
Epoch 316/5000
1/1 [==============================] - 0s 11ms/step - loss: 208.0861 - val_loss: 211.0889
Epoch 317/5000
1/1 [==============================] - 0s 11ms/step - loss: 206.5808 - val_loss: 208.6415
Epoch 318/5000
1/1 [==============================] - 0s 11ms/step - loss: 205.0973 - val_loss: 206.2136
Epoch 319/5000
1/1 [==============================] - 0s 11ms/step - l

...

1/1 [==============================] - 0s 11ms/step - loss: 100.5026 - val_loss: 0.3798
Epoch 784/5000
1/1 [==============================] - 0s 11ms/step - loss: 100.5004 - val_loss: 0.3746
Epoch 785/5000
1/1 [==============================] - 0s 11ms/step - loss: 100.4981 - val_loss: 0.3694
Epoch 786/5000
1/1 [==============================] - 0s 12ms/step - loss: 100.4958 - val_loss: 0.3643
Epoch 787/5000
1/1 [==============================] - 0s 11ms/step - loss: 100.4938 - val_loss: 0.3593
Epoch 788/5000
1/1 [==============================] - 0s 11ms/step - loss: 100.4917 - val_loss: 0.3543
Epoch 789/5000
1/1 [==============================] - 0s 11ms/step - loss: 100.4896 - val_loss: 0.3494
Epoch 790/5000
1/1 [==============================] - 0s 11ms/step - loss: 100.4874 - val_loss: 0.3446
Epoch 791/5000
1/1 [==============================] - 0s 11ms/step - loss: 100.4855 - val_loss: 0.3399
Epoch 792/5000
1/1 [==============================] - 0s 11ms/step - loss: 100.4835 - va

1/1 [==============================] - 0s 11ms/step - loss: 100.3945 - val_loss: 0.1267
Epoch 863/5000
1/1 [==============================] - 0s 11ms/step - loss: 100.3938 - val_loss: 0.1249
Epoch 864/5000
1/1 [==============================] - 0s 11ms/step - loss: 100.3931 - val_loss: 0.1232
Epoch 865/5000
1/1 [==============================] - 0s 11ms/step - loss: 100.3923 - val_loss: 0.1215
Epoch 866/5000
1/1 [==============================] - 0s 11ms/step - loss: 100.3916 - val_loss: 0.1198
Epoch 867/5000
1/1 [==============================] - 0s 11ms/step - loss: 100.3909 - val_loss: 0.1182
Epoch 868/5000
1/1 [==============================] - 0s 12ms/step - loss: 100.3900 - val_loss: 0.1166
Epoch 869/5000
1/1 [==============================] - 0s 11ms/step - loss: 100.3895 - val_loss: 0.1149
Epoch 870/5000
1/1 [==============================] - 0s 11ms/step - loss: 100.3888 - val_loss: 0.1134
Epoch 871/5000
1/1 [==============================] - 0s 11ms/step - loss: 100.3882 - va

1/1 [==============================] - 0s 11ms/step - loss: 100.3585 - val_loss: 0.0423
Epoch 942/5000
1/1 [==============================] - 0s 11ms/step - loss: 100.3581 - val_loss: 0.0417
Epoch 943/5000
1/1 [==============================] - 0s 11ms/step - loss: 100.3580 - val_loss: 0.0411
Epoch 944/5000
1/1 [==============================] - 0s 11ms/step - loss: 100.3578 - val_loss: 0.0405
Epoch 945/5000
1/1 [==============================] - 0s 11ms/step - loss: 100.3575 - val_loss: 0.0400
Epoch 946/5000
1/1 [==============================] - 0s 11ms/step - loss: 100.3571 - val_loss: 0.0394
Epoch 947/5000
1/1 [==============================] - 0s 11ms/step - loss: 100.3571 - val_loss: 0.0389
Epoch 948/5000
1/1 [==============================] - 0s 11ms/step - loss: 100.3568 - val_loss: 0.0383
Epoch 949/5000
1/1 [==============================] - 0s 11ms/step - loss: 100.3566 - val_loss: 0.0378
Epoch 950/5000
1/1 [==============================] - 0s 11ms/step - loss: 100.3562 - va

1/1 [==============================] - 0s 11ms/step - loss: 100.3465 - val_loss: 0.0141
Epoch 1021/5000
1/1 [==============================] - 0s 11ms/step - loss: 100.3464 - val_loss: 0.0139
Epoch 1022/5000
1/1 [==============================] - 0s 11ms/step - loss: 100.3463 - val_loss: 0.0137
Epoch 1023/5000
1/1 [==============================] - 0s 11ms/step - loss: 100.3461 - val_loss: 0.0135
Epoch 1024/5000
1/1 [==============================] - 0s 11ms/step - loss: 100.3462 - val_loss: 0.0133
Epoch 1025/5000
1/1 [==============================] - 0s 11ms/step - loss: 100.3461 - val_loss: 0.0131
Epoch 1026/5000
1/1 [==============================] - 0s 11ms/step - loss: 100.3460 - val_loss: 0.0130
Epoch 1027/5000
1/1 [==============================] - 0s 11ms/step - loss: 100.3459 - val_loss: 0.0128
Epoch 1028/5000
1/1 [==============================] - 0s 11ms/step - loss: 100.3459 - val_loss: 0.0126
Epoch 1029/5000
1/1 [==============================] - 0s 11ms/step - loss: 100.

Epoch 1099/5000
1/1 [==============================] - 0s 11ms/step - loss: 100.3425 - val_loss: 0.0047
Epoch 1100/5000
1/1 [==============================] - 0s 11ms/step - loss: 100.3425 - val_loss: 0.0046
Epoch 1101/5000
1/1 [==============================] - 0s 11ms/step - loss: 100.3424 - val_loss: 0.0046
Epoch 1102/5000
1/1 [==============================] - 0s 11ms/step - loss: 100.3422 - val_loss: 0.0045
Epoch 1103/5000
1/1 [==============================] - 0s 11ms/step - loss: 100.3424 - val_loss: 0.0044
Epoch 1104/5000
1/1 [==============================] - 0s 11ms/step - loss: 100.3424 - val_loss: 0.0044
Epoch 1105/5000
1/1 [==============================] - 0s 12ms/step - loss: 100.3423 - val_loss: 0.0043
Epoch 1106/5000
1/1 [==============================] - 0s 12ms/step - loss: 100.3423 - val_loss: 0.0043
Epoch 1107/5000
1/1 [==============================] - 0s 12ms/step - loss: 100.3423 - val_loss: 0.0042
Epoch 1108/5000
1/1 [==============================] - 0s 12ms/s

In [19]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 44ms/step
[542.03](test_target) - [[500.1869]](prediction) = 41.84311035156247


In [20]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [21]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,Tocantins - PIB - Estadual,Tocantins - PIB - Construção Civil,Tocantins - PIB - Per Capita,Tocantins - PIB - Preços de Mercado,Tocantins - IDH,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),Contratação SFH (Bi R$),...,Tocantins - Produção de Cimento (t),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Tocantins - IDH Renda,Tocantins - Desemprego
0,-1.295415,-1.127439,-1.295974,-1.229472,-1.413058,1.165029,1.396038,1.335266,-1.330895,1.406550,...,0.664999,0.288160,0.867298,-1.339838,-1.201022,1.165594,1.413752,-0.726293,-1.413602,-1.326537
1,0.156357,-0.175646,0.157743,0.009510,0.657035,0.111769,-0.502296,-0.264151,0.251276,-0.575945,...,-1.413394,1.054971,0.533742,0.277969,1.247164,0.110775,-0.738152,1.414038,0.742810,1.087797
2,1.139058,1.303085,1.138231,1.219962,0.756024,-1.276798,-0.893743,-1.071116,1.079619,-0.830605,...,0.748395,-1.343131,-1.401040,1.061869,-0.046142,-1.276370,-0.675601,-0.687746,0.670793,0.238740


train_target:


,Tocantins - Consumo de Cimento (t)
0,454.929429
1,489.016286
2,500.270000


test_input:


...

,Tocantins - PIB - Estadual,Tocantins - PIB - Construção Civil,Tocantins - PIB - Per Capita,Tocantins - PIB - Preços de Mercado,Tocantins - IDH,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),Contratação SFH (Bi R$),...,Tocantins - Produção de Cimento (t),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Tocantins - IDH Renda,Tocantins - Desemprego
6,-0.959434,1.618038,-0.62215,1.475914,-0.117545,1.16956,2.035192,0.985175,-0.705937,2.048732,...,1.948593,-1.306124,-1.803231,-0.345228,-0.098527,-0.452549,-1.149265,-2.063869,-0.097313,-1.529923


test_target:


,Tocantins - Consumo de Cimento (t)
6,542.03


Epoch 1116: early stopping
1/1 [==============================] - 0s 31ms/step
Error: 41.84311035156247




[494.6426086425781, 389.4751892089844, 433.21270751953125, 500.1868896484375]

In [22]:
display(targets)
display(predictions)

[396.438, 402.932, 500.126, 542.03]

[494.6426086425781, 389.4751892089844, 433.21270751953125, 500.1868896484375]

In [23]:
mae = mean_absolute_error(predictions, targets)
mae

55.104455566406244

In [24]:
porcentage = mae/np.mean(targets)
porcentage

0.1196930275573763